<a href="https://colab.research.google.com/github/ryry311/482FinalProject/blob/master/upenn_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import time, json 
from bs4 import BeautifulSoup
import nltk 
import os

nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')

from nltk import pos_tag, word_tokenize,sent_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
def eventExtractor(html_string):
    response = requests.get(html_string)
    soup = BeautifulSoup(response.content, "html.parser")

    contentBlock = soup.find_all("article")
    
    #missing bolded "updated" and "deadline for submission" text 

    event = [] 
    for article in contentBlock:
        rawText = '' 
        
        for title in (article.find_all("h2", {"class" : "node-title"})):
            if(title.text.strip() == ""):
                continue
            rawText += title.text.strip()+'\n'

        for loc in (article.find_all("div", {"class" : "field-items"})): 
            if(loc.text.strip() == ""):
                continue
            rawText += loc.text.strip()+'\n'
        
        event.append(rawText)

    return event 

In [0]:
def upenn_json(dataTable):
    upenn_event_json = [] 

    for entry in dataTable: 
        entry_dict = {}
        meta_data = entry.split("\n")

        title  = meta_data[0]
        publisher_and_location = meta_data[2]
        submission_deadline = meta_data[3]

        separator = " ,"
        description = separator.join(meta_data[4:-1])

        entry_dict['title'] = title
        entry_dict['publisher_and_location'] = publisher_and_location
        entry_dict['submission_deadline'] = submission_deadline
        entry_dict['description'] = description

        upenn_event_json.append(entry_dict)

    return upenn_event_json

In [6]:

data = [] 

for x in range(20):
    url = "https://call-for-papers.sas.upenn.edu/category/all/backtrack?page={}".format(x)
    print(url)
    pageEvent = eventExtractor(url)
    data += (pageEvent)





https://call-for-papers.sas.upenn.edu/category/all/backtrack?page=0
https://call-for-papers.sas.upenn.edu/category/all/backtrack?page=1
https://call-for-papers.sas.upenn.edu/category/all/backtrack?page=2
https://call-for-papers.sas.upenn.edu/category/all/backtrack?page=3
https://call-for-papers.sas.upenn.edu/category/all/backtrack?page=4
https://call-for-papers.sas.upenn.edu/category/all/backtrack?page=5
https://call-for-papers.sas.upenn.edu/category/all/backtrack?page=6
https://call-for-papers.sas.upenn.edu/category/all/backtrack?page=7
https://call-for-papers.sas.upenn.edu/category/all/backtrack?page=8
https://call-for-papers.sas.upenn.edu/category/all/backtrack?page=9
https://call-for-papers.sas.upenn.edu/category/all/backtrack?page=10
https://call-for-papers.sas.upenn.edu/category/all/backtrack?page=11
https://call-for-papers.sas.upenn.edu/category/all/backtrack?page=12
https://call-for-papers.sas.upenn.edu/category/all/backtrack?page=13
https://call-for-papers.sas.upenn.edu/catego

In [0]:
upenn_json = upenn_json(data)
upenn_json_obj = json.dumps(upenn_json)

with open("uppen_events.json", "w") as outfile: 
    outfile.write(upenn_json_obj) 